In [2]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report
from keras.utils.np_utils import to_categorical
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation
from keras.regularizers import L1L2
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import Callback
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import feather
import scikitplot as skplt
import matplotlib.pyplot as plt
from datetime import datetime

print("Done!")

Using TensorFlow backend.


Done!


In [3]:
df = pd.read_csv('SBA_All_Extra_Data.csv')
df['LoanStatus'].replace('CHGOFF',1, inplace=True)
df['LoanStatus'].replace('PIF',0, inplace=True)

#Note: It is recommended to not change any of these parameters for performance improvement.
#The only value that should be changed between runs is the year_thresh variable to see how
#default probability modeling varies over different timespans.


year_thresh = 5
total = 0
for i in range(len(df)):
    if df.LoanStatus[i]  == 1:
        start = datetime.strptime( df.loc[i, 'ApprovalDate'], "%m/%d/%Y")
        end = datetime.strptime( df.loc[i, 'ChargeOffDate'], "%m/%d/%Y")
        delta = (end - start).days
        if delta / 365 > year_thresh:
            df.loc[i, 'LoanStatus'] = 0
        total += df.loc[i, 'LoanStatus']
print(total)


df['NaicsCode'].fillna( 0, inplace=True)
ThirdPartyDollars_mean = df['ThirdPartyDollars'].mean(skipna = True)
df['ThirdPartyDollars'].fillna(ThirdPartyDollars_mean, inplace = True)
df['ThirdPartyLender_State'].fillna('Unknown', inplace = True)
df['SectorCode'] = df['NaicsCode'].apply(lambda x: int(x*1e-4))


C:\Users\Brian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (12,13,14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


3436
Done!


In [68]:
for i in range(len(df.columns)):
    print(df.columns[i])

Unnamed: 0
Program
BorrName
BorrStreet
BorrCity
BorrState
BorrZip
CDC_Name
CDC_Street
CDC_City
CDC_State
CDC_Zip
ThirdPartyLender_Name
ThirdPartyLender_City
ThirdPartyLender_State
ThirdPartyDollars
GrossApproval
ApprovalDate
ApprovalFiscalYear
DeliveryMethod
subpgmdesc
InitialInterestRate
TermInMonths
NaicsCode
NaicsDescription
ProjectCounty
ProjectState
BusinessType
LoanStatus
ChargeOffDate
GrossChargeOffAmount
ag_Public_nom
ag_Private_nom
ag_Public_dollar
ag_Private_dollar
ag_Total
cci_CCI
cci_CCI_c
creative_Metro_93
creative_Metro_03
creative_Employed
creative_Creative
creative_Arts
creative_Creative_frac
creative_Arts_frac
farm_Num_Farms_Supply
farm_NumShares
farm_CSA_organic
income_n1_1
income_prep_1
income_n2_1
income_a00100_1
income_a00200_1
income_a00300_1
income_a00600_1
income_a00900_1
income_n1_2
income_prep_2
income_n2_2
income_a00100_2
income_a00200_2
income_a00300_2
income_a00600_2
income_a00900_2
income_a18300_2
income_a19700_2
income_n1
income_prep
income_n2
income_a001

In [7]:
col_start = df.columns.get_loc('income_n1_1')
col_end = df.columns.get_loc('income_a19700_6')
df_zip_columns = list(df.columns[col_start:col_end+1])
for col in df_zip_columns:
    df[col].fillna( df[col].mean(), inplace=True) 
col_start = df.columns.get_loc('ed_cty_ENROLL')
col_end = df.columns.get_loc('ed_cty_TCAPOUT')
df_ed_columns_1 = list(df.columns[col_start:col_end+1])
for col in df_ed_columns_1:
    df[col].fillna( df[col].mean(), inplace=True)
col_start = df.columns.get_loc('ed_st_ENROLL')
col_end = df.columns.get_loc('ed_st_CAPITAL_OUTLAY_EXPENDITURE')
df_ed_columns_2 = list(df.columns[col_start:col_end+1])
for col in df_ed_columns_2:
    df[col].fillna( df[col].mean(), inplace=True)    
    
df['farm_Num_Farms_Supply'].fillna( 0, inplace=True)
df['farm_NumShares'].fillna( 0, inplace=True)
df['farm_CSA_organic'].fillna( 0, inplace=True)
df['unemp_m_value'].fillna(df['unemp_m_value'].mean(), inplace=True)
df['unemp_Rate'].fillna(df['unemp_Rate'].mean(), inplace=True)
df['zillow_SizeRank'].fillna(df['zillow_SizeRank'].mean(), inplace=True)
df['zillow_SizeRank.1'].fillna(df['zillow_SizeRank.1'].mean(), inplace=True)
df['zillow_SizeRank.2'].fillna(df['zillow_SizeRank.2'].mean(), inplace=True)
df['zillow_SizeRank.3'].fillna(df['zillow_SizeRank.3'].mean(), inplace=True)
df['zillow_PctIncrease'].fillna(df['zillow_PctIncrease'].mean(), inplace=True)
df['zillow_PctDecrease'].fillna(df['zillow_PctDecrease'].mean(), inplace=True)
df['zillow_MedianValue'].fillna(df['zillow_MedianValue'].mean(), inplace=True)
df['zillow_ZHVI'].fillna(df['zillow_ZHVI'].mean(), inplace=True)
df['SP_Adj_Close'].interpolate(method='nearest', inplace=True)
df['SP_Returns'].fillna(df['SP_Returns'].mean(), inplace=True)
df['ag_Private_dollar'].fillna(df['ag_Private_dollar'].mean(), inplace=True)
df['ag_Private_nom'].fillna(df['ag_Private_nom'].mean(), inplace=True)
df['ag_Public_dollar'].fillna(df['ag_Public_dollar'].mean(), inplace=True)
df['ag_Public_nom'].fillna(df['ag_Public_nom'].mean(), inplace=True)
df['patent_Patent'].fillna(df['patent_Patent'].mean(), inplace=True)
df['patent_Patent_frac'].fillna(df['patent_Patent_frac'].mean(), inplace=True)
df['cci_CCI'].fillna(df['cci_CCI'].mean(), inplace=True)


print("Done!")

Done!


In [71]:
df_borr = pd.get_dummies(df.ProjectState, prefix = 'Borr') 
df_lend = pd.get_dummies(df.ThirdPartyLender_State, prefix = 'Lend') 
df_cdc = pd.get_dummies(df.CDC_State, prefix = 'CDC') 
df_year = pd.get_dummies(df.ApprovalFiscalYear, prefix = 'appr') 
df_buis = pd.get_dummies(df.BusinessType, prefix = 'buis') 
df_sect = pd.get_dummies(df.SectorCode, prefix = 'sect') 
df_term = pd.get_dummies(df.TermInMonths, prefix = 'term') 
df_income = df[df_zip_columns] 
df_income = (df_income - df_income.min())/(df_income.max() - df_income.min()) 
df_ed_1 = df[df_ed_columns_1] 
df_ed_2 = df[df_ed_columns_2] 
data = df_sect 
data = pd.concat([data, df_year], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df_buis], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.ThirdPartyDollars)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.GrossApproval)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df_term], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.SP_Adj_Close + 1)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.bci_BCI)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.cci_CCI)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df_lend], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df_borr], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df_cdc], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.ag_Public_nom)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.ag_Private_nom)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.ag_Public_dollar)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.ag_Private_dollar)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df.farm_Num_Farms_Supply], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df.farm_CSA_organic], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df.farm_NumShares], axis=1, join_axes=[data.index])


data = pd.concat([data, df.unemp_m_value], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df.unemp_Rate], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df_income + 1)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df_ed_1 + 1)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df_ed_2 + 1)], axis=1, join_axes=[data.index])
data = pd.concat([data, np.log(df['zillow_ZHVI'] + 1)], axis=1, join_axes=[data.index])
data = pd.concat([data, np.log(df['zillow_SizeRank'] + 1)], axis=1, join_axes=[data.index])
data = pd.concat([data, np.log(df['zillow_SizeRank.1'] + 1)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df['zillow_SizeRank.2'] + 1)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df['zillow_SizeRank.3'] + 1)], axis=1, join_axes=[data.index]) 
data = pd.concat([data, np.log(df.zillow_MedianValue + 1)], axis=1, join_axes=[data.index])

data = (data-data.min())/(data.max()-data.min())

#Do not normalize the data corresponding to potentially negative percentages 
data = pd.concat([data, df.zillow_PctDecrease], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df.zillow_PctIncrease], axis=1, join_axes=[data.index]) 
data = pd.concat([data, df.SP_Returns], axis=1, join_axes=[data.index])

#Normalize data using max to min to bring values between 0 and 1. This will help learn faster for elu 
ordered_data = data 
data = data.as_matrix() 
labels = df['LoanStatus'].as_matrix() 
truth = labels

In [72]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [73]:
data, labels = unison_shuffled_copies(data, labels)

training_percentage = 0.92
validation_percentage = 0.04
cutoff = int(training_percentage*len(data))

X_train = data[0:cutoff][:]
Y_train = labels[0:cutoff]

cutoff_2 = int((training_percentage+validation_percentage)*len(data))
X_val = data[cutoff + 1:cutoff_2][:]
Y_val = labels[cutoff + 1:cutoff_2]
X_test = data[cutoff_2 + 1:][:]
Y_test = labels[cutoff_2 + 1:]
X_train = X_train.astype('float32') 
X_val = X_val.astype('float32') 
X_test = X_test.astype('float32')

#Switch between NN and basic Sigmoid function by setting this flag. 
#Note: Update Hyperparameters accordingly for better performance

implementNN = True

#Note: mode 1 is a deep model whereas mode 2 is less deep and even more shallow for mode 3. This can be adjusted as you like.
#elu are ideal for the normalized data
mode = 2

model = Sequential() 
num_features = data.shape[1]
if implementNN  and mode == 1:
    
    model.add(Dense(32, input_dim=num_features, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.2))

    model.add(Dense(64, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
 
    model.add(Dense(128, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.1))

    model.add(Dense(256, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(512, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(1024, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(2048, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    #model.add(Dropout(0.5))
    
    model.add(Dense(1024, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    #model.add(Dropout(0.5))
    
    model.add(Dense(512, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(128, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.4))

    model.add(Dense(64, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.25))

    model.add(Dense(32, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(1, init='normal'))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    
elif implementNN and mode == 2 :
    
    model.add(Dense(32, input_dim=num_features, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(64, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
 
    model.add(Dense(128, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.2))

    model.add(Dense(256, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(512, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.3))    
    
    model.add(Dense(256, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(128, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.2))

    model.add(Dense(64, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.2))

    model.add(Dense(32, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(output_dim = 1, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0)))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))

elif implementNN and mode == 3 :
    
    model.add(Dense(32, input_dim=num_features, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(64, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
 
    model.add(Dense(128, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(256, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(128, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0.0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(64, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Dense(32, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    
    model.add(Dense(output_dim = 1, init='normal', kernel_regularizer=L1L2(l1=0.0, l2=0)))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))

else :
    #Note : One layer Neural Network is the same as a logit model if sigmoid is activation
    model.add(Dense(output_dim = 1, kernel_regularizer=L1L2(l1=0.0, l2=0.0),input_dim=num_features, activation='sigmoid',  init='normal'))
    
    

C:\Users\Brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, input_dim=398, kernel_regularizer=<keras.reg..., kernel_initializer="normal")`
C:\Users\Brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_regularizer=<keras.reg..., kernel_initializer="normal")`
C:\Users\Brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:109: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg..., kernel_initializer="normal")`
C:\Users\Brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_regularizer=<keras.reg..., kernel_initializer="normal")`
C:\Users\Brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:119: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_regularizer=<keras.reg..

In [74]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [ ]:
batch_size = 128
epochs = 200
opt = optimizers.Adagrad(lr=0.08)
model.summary()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['accuracy', auc_roc]) 
# 
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), verbose = 1,epochs = epochs, batch_size = batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 32)                12768     
_________________________________________________________________
batch_normalization_61 (Batc (None, 32)                128       
_________________________________________________________________
activation_61 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_62 (Batc (None, 64)                256       
_________________________________________________________________
activation_62 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 128)               8320      
__________

In [ ]:
for layer in model.layers:
    weights = (layer.get_weights()) # list of numpy arrays

score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])
print('Test ROC', score[2])
#weights = weights[0]

Y_pred = model.predict_proba(X_train)
print(roc_auc_score(Y_train, Y_pred))

Y_pred = model.predict_proba(X_test)
print(roc_auc_score(Y_test, Y_pred))

full_data = ordered_data.as_matrix()
ordered_probabilities = model.predict_proba(full_data)
print(roc_auc_score(truth, ordered_probabilities))
df['default_probabilities'] = ordered_probabilities
df['LoanStatus'].replace(1, 'CHGOFF', inplace=True)
df['LoanStatus'].replace(0, 'PIF', inplace=True)
for i in range(len(ordered_probabilities)):
    print(ordered_probabilities[i], " ", df.LoanStatus[i])
df.to_csv("SBA_Data_with_Default_Probabilities_" + str(year_thresh), sep=',')

In [ ]:
fpr, tpr, threshold = roc_curve(df['LoanStatus'], df['default_probabilities'])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, 'g', label = ' 8 Layer Network: AUC = %0.4f' % roc_auc)

plt.title('Receiver Operating Characteristic for General Probability of Default')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
